Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 2

## Introduction to question 2

In the second question of this assignment, we will explore the use of local search in genome assembly.

We will use local search to assemble (construct) a large part of the nucleotide sequence of the monkeypox virus, which has been downloaded from the National Center for Biotechnology Information in the United States. Please note that no additional or specialized knowledge of biology or bioinformatics is required for this assignment. (Actechnical specifics of bioinformatics have been adapted and simplified for the purposes of this computer science assignment, so if you are a biologist, please do not apply preexisting knowledge to solve the problem. Furthermore, you should not attempt to search up the genome on genomic databases to "guess" the actual sequence, since we are more interested in your coding methodology rather than your attempts at reproducing a known sequence.)

This is an introductory computer science assignment and not a bioinformatics assignment; we are simply using bioinformatics as a use case to illustrate the applicability of local search to the natural sciences. Therefore, no knowledge of bioinformatics is assumed or required. In the paragraphs that follow, I will give a short crash course which will cover all the domain knowledge you will need to know in order to tackle this problem.  

For technical reasons, when we analyze the nucleotide sequence (genome) of a virus, we usually cannot “read” it in one fell swoop. We have to read the genome in parts, because the genome is usually too long for the machine to read in a single sitting. To simplify things, a “read” is a single view of part of the genome; think of it as a SUBSTRING, a partial view of the whole genome. After we have generated multiple reads of a genome, we then have to “stitch”, or combine, the different reads of the genome together. This process of stitching up reads of a genome into the final sequence is known as genome assembly. However, the different reads of the genome cannot just be concatenated like usual string concatenation. It’s not a situation where you have one read, “Hello”, and another read, “World”, and all you need to do is concatenate both strings together to make “Hello World”. Among other reasons, there are two major reasons why you can’t do so:

1. You do not know which read came first. The reads are not ordered. How do you know “Hello” came after “World”? The answer is that you don’t. Imagine how complicated this situation might be if you had more than two reads. (This is indeed our situation, where we have $n$ reads, and $n>>2$.)

2. One read may contain a substring contained in another read. Specifically, without loss of generality, part of the ending $x$ characters of a read (i.e., suffix) might also be found in the starting $x$ positions (i.e., prefix) of another read.

- A computer scientist usually creates opportunities from problems. While this may be a “problem” in that you just can’t concatenate two strings blindly, the fact that strings contain shared “substrings” is actually a very helpful clue that you can use to “join” strings together. 

- Note that the choice of the value of $x$ could be a hyperparameter decided by the computer scientist.

## Your tasks

In this part of the assignment, you will work with (simulated) reads that I have generated from the nucleotide sequence of the monkeypox virus. In reality, bioinformatics is far more complicated, but here we will work with a simplified situation. Your task is to examine the reads that I have provided for you, and from there “infer” the nucleotide sequence that might have produced those reads. 

The reads are provided in the csv file `data.csv` which simply provides a list of unique strings. Note that you should NOT assume any particular ordering of the strings in this dataframe. In fact, the strings have already been shuffled randomly. 

NOTE: You are not allowed to use `pandas` or any other libraries apart from the Python STL to load the csv file.

### Task A (3 marks): 

Create a directed graph. The nodes in the graph are the strings in the list of reads. An edge should be drawn FROM read A TO read B if and only if a suffix (of length $x$) of read A is also a prefix (obviously, also of length $x$) of read B. For the purposes of the assignment, limit the value of $x$ to between 5 and 30, both inclusive. That is, to be clear, $5\leq x\leq 30$. The weight of an edge between read A and read B should be the NEGATED value of $x$, i.e. $-x$. 

In your Jupyter notebook, please report the number of edges in your graph. Provide a barplot or histogram which shows the number of edges with different weights or weight categories. In this task, you are free to use plotting libraries such as `matplotlib` or `seaborn` to plot this graph.

As an example, if read A is "TACTAGT" and read B is "TAGTCCCCT", then an edge is drawn FROM read A TO read B (i.e., $A \rightarrow B$) with weight of $-4$. This is because the 4-suffix "TAGT" is also the 4-prefix of read B; in other words, the last 4 characters of read A (a substring of length 4) overlap with the first 4 characters of read B (a substring of length 4).

### Task B (7 marks): 

From Task A, you now have a graph which shows connections between reads based on how they overlap, in theory you could draw a path through the graph and thereby derive the full sequence (genome).

Task B asks you to use local search method(s) to determine a path through this directed graph of strings. 

- You are expected to use simulated annealing and tune the relevant configuration settings and hyperparameters. The minimum requirement is to implement simulated annealing.

- Explain tha rationale behind the choice of scheduling strategy and parameters.

- However, you may also explore other search methods in addition to simulated annealing. Marks will be awarded for effort.

Note the following constraints:

1. The path has to go through each and every vertex exactly once. For computer scientists, this constraint is reminiscent of the "Traveling Salesman's Problem", except that unlike TSP, we should not need to go back to the starting vertex again. 

2. For the purposes of neighbor generation / action selection at each node, bear in mind that a path through the graph which minimizes the total number of nucleotides in the assembled sequence is the preferred path. To state that another way, the assembled sequence should be derived from a path that goes through EACH and EVERY vertex exactly once, however we want this assembled sequence to be AS SHORT AS POSSIBLE.

3. You are not given the starting (source/origin) or ending (destination) vertex.

4. For avoidance of ambiguity, no cycles are allowed. You must not visit a vertex more than once.

5. You are not allowed to use any libraries apart from the Python Standard Library.
No import statements which import libraries outside of the Python STL should be found within your answer for Task B.

Please remember to report the assembled sequence that you obtain. Although it would be great if you can come up with a good sequence, please feel reassured that we are more interested in your APPROACH to the problem, and so you can potentially get a reasonable score on this task even if your solution is "wrong". It is the process, rather than the result, which matters more.

## Enironment Setup

In [1]:
import os
os.chdir(r"C:\Users\e0833561\OneDrive - National University of Singapore\Desktop\it5005-proj-1\AI-searching-team-proj")
print(os.getcwd())

C:\Users\e0833561\OneDrive - National University of Singapore\Desktop\it5005-proj-1\AI-searching-team-proj


## Ingest data

In [ ]:

import csv

dna_fragments_list = []
with open('./data/data.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            # print(f'Column names are {", ".join(row)}')
            pass
            line_count += 1
        else:
            # print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            dna_fragments_list.append(row[2])
            # line_count += 1
    print(f'{dna_fragments_list[:2]}')


: 

## Task A outline

### Pseudo-algorithm
1. Start from arbitrary state
    - Simply start from read order
2. Nodes are `dna_fragments`.
3. Can think of directed edges as pointing towards the next node, or in the words of the hint, from postfix to prefix.
4. Initialize directed edges from $i$ to $i + 1$.
5. Relax the constraint that $5 <= x <= 30$, where $x$ is number of letters that overlap.
    - Constraint becomes $0 <= x$ which is satisfied by any graph.
6. Think of actual cost we are trying to minimize as the total lengths of the joined substring, which is the sum of lengths of individual segments minum sum of lengths of overlaps (count each overlap only once).
7. For the "random flip" analogy of crystallization, we do the following:
    1. Generate a candidate change 
        1. Check if constraint satisfied. If not, generate another candidate change.
    2. Check if heuristic cost of sum of $x$ decreases.
        - if cost decreases: accept
        - if cost does not decrease, accept with probability given by the temperature equation
    


### Implementation Specific Tasks
0. I Think the framework of `class Problem` is couched in the terminology of an uninformed tree search (tree search in the sense we do not have to check for whether node is reached before.). But in this framework, the node is the entire state (includes both node of `dna_fragment` and edges of dna fragment overlaps.)
1.In `__init__()`. initialize state we wish to keep track of.
    - Nodes: list of strings representing DNA fragments.
        - Use enumerate to generate id automatically. Treat the list as immutable.
    - Edges: 
        - Option 1: list of tuples in the format (from_index, to_index, x) like adjacency list in graph algorithms
        - Option 2: just join them up with double-linked lists, with the next pointer as the one referred to in the notebook hint, but the backlink for tracking predecessor.
        - Suppose we go with Option 2 for simplicity and explicitness.
2. In `actions`, we generate candidate change. 


In [ ]:
# Problem Class
class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

: 

In [ ]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 

: 

In [ ]:
# Code to generate neighbours, value of states, etc.
class TSP(Problem):
    def __init__(self, graph, csvpath, initial=None, goal=None, **kwds):
        super().__init__(initial, goal, **kwds)
        self.graph= graph
        self.csvpath= csvpath
        self.initial= self.generate_initial_state(self.graph)

    
    
    def generate_initial_state(self,graph_dictionary):
        # This function takes graph dictionary, and returns a complete tour with greedy selection
        initial_state=[]
    
        #start with read 0
        current_read= 0
        reached_read=[]

        initial_state.append(current_read)
        reached_read.append(current_read)

        while True:

            #create temp list with format [(224, -16),(302, -5)], where current read can go to read 224 with -16 weight, 302 with -5 weight, etc       
            temp_list=[]

            # Cycle through all connections to a read, and output in a temporary list
            for key in graph_dictionary:
                if key[0]== current_read:
                # cycle through all weights (if more than 1) between 1 edge 
                    for i in graph_dictionary[key]:
                        temp_list.append((key[1],i))


            #*****TRACING******
            print(temp_list)


            # Out of all the edges, find the minimum edge in a greedy way for the initial stage. 
            min_weight=((0,10000))

            for element in temp_list:
                # Check if the next read is not already reached. 
                if element[0] not in reached_read:
                    if element[1]<min_weight[1]:
                        min_weight=((element[0],element[1]))

            # If a read has no other neighbour, go to the first unreached read with weight 0.
            if min_weight[0]==0 and min_weight[1]==10000:
                for i in range(0, number_of_reads):
                    if i not in reached_read:
                        min_weight=(i,0)
                        break

            #*****TRACING******        
            print(f"Read {current_read} goes to {min_weight[0]} with weight {min_weight[1]}")

            initial_state.append(min_weight[0])
            reached_read.append(min_weight[0])
            temp_list.clear()        

            # Check if we reached all elements, if yes, stop the loop. If no, continue loop
            if len(reached_read)==number_of_reads:
                break
            else:
                current_read=min_weight[0]


        print(f"The initial state is {initial_state}")

        print(initial_state)
                
        return initial_state

        


    def get_weight_between_two_reads(self,testing_tuple):
        # This function checks the graph for the weights between 2 reads, and takes the most negative
        if testing_tuple in self.graph.keys():
            setofweights= self.graph[testing_tuple]
            #There could be more than 1 edge between 2 reads, pick most negative
            return min(setofweights)
        else:
            #If there is no edge between 2 reads, assume weight is 0
            return 0
        
        
    
    def value_of_state(self,state):
        # This function takes a state- a complete tour eg [1,20,3,5,7,9,200...]
        # and returns the value of the entire tour, which is the sum of all the weights
        # This is done by checking the graph for the weights between 2 reads, and taking the most negative
        
        valuecounter=0 
        
        for i in range(len(state)):
            if i+1<len(state):
                testing_tuple=((state[i],state[i+1]))
                valuecounter+=self.get_weight_between_two_reads(testing_tuple)
        
        return valuecounter
            

        
        
    def generate_genome_sequence(self,state):
        # This function takes a state- a complete tour eg [1,20,3,5,7,9,200...]
        # and returns a) The complete stitched string, with overlaps merged
        # and b) The length of the completed stitched string

        temparray=[]
        genome_sequence=""
    
        with open(self.csvpath) as f:
            f.readline()
            counter=0
            for line in f:
                list_form=line.rstrip('\n').split(',')
                temparray.append([list_form[0]])
                temparray[counter].append(list_form[2])
                counter+=1
               
        for i in range(len(state)):
            if i+1<len(state):
                testing_tuple=((state[i],state[i+1]))

                weight=self.get_weight_between_two_reads(testing_tuple)

                if weight!=0:
                    genome_sequence+=temparray[state[i]][1][:weight]
                else:
                    genome_sequence+=temparray[state[i]][1]
                    
                
        return genome_sequence, len(genome_sequence)
    
    
    
    
    
    def generate_neighbour(self,state):
        # This function takes a state- a complete tour eg [1,20,3,5,7,9,200...]
        # It goes through the 2-Opt method
        # and returns a neighbouring state- another complete tour, eg [1,20,9,5,7,...]
        

        #TODO
        pass
    
    
    
    


: 

In [ ]:
# STEP 1: GENERATE THE GRAPH AND INITIALISE THE PROBLEM 


# 1a: Function for taking in the CSV file, and output graph in dictionary 
# Dictionary has a format of example {(0, 1):{-20,-5-6}}, where 0 has an edge to 1, with weights of -20, -5 and -6. 
csvpath='q2 testing/dummy.csv'

def readcsvfile(filename):
    temparray=[]
    outputgraph={}
    
    with open(filename) as f:
        f.readline()
        counter=0
        for line in f:
            list_form=line.rstrip('\n').split(',')
            temparray.append([list_form[0]])
            temparray[counter].append(list_form[2])
            counter+=1
    
    for i in range(len(temparray)):
        currentindex=temparray[i][0]
        currentreadsequence=temparray[i][1]
        
        for m in range(len(temparray)):
            if i==m:
                continue
  
            for k in range(2,20):
                testingreadsequence=temparray[m][1]

                if currentreadsequence[-k:]==testingreadsequence[0:k]:
                    if outputgraph.get((i,m),'error not found') != 'error not found':
                        outputgraph[(i,m)].add(-k)
                        
                    else:
                        outputgraph[(i,m)]={-k}

    #print(counter)
    return(outputgraph, counter)
    
graph_dictionary, number_of_reads= readcsvfile(csvpath)
print(graph_dictionary)



# 1b: Initialise the Problem TSP Class with the graph, and the path to the CSV (used for stitching at the end).
# Initial state is immediately generated by the generate_initial_state method in the TSP Class

tsp=TSP(graph_dictionary, csvpath)



# 1c: Test and check the value of initial state. 

value_of_state= tsp.value_of_state(tsp.initial)

print(f"The value of the initial state is {value_of_state}")



# 1d: Test and check the sequence created by the initial state. 

stitched_string, length_stitched= tsp.generate_genome_sequence(tsp.initial)
print("")
print("")
print(f"The length of the stitched genone of the initial state is {length_stitched}")
print(f"The stitched genome is")
print(stitched_string)




      
      

: 

In [ ]:
# STEP 2: START SIMULATED ANNEALING







: 

: 

: 

: 

: 